**Weather Checker Project**

Handle Settings

In [1]:
import json

settings = {}

def load_settings():
  global settings
  try:
    with open('settings.json', 'r') as f:
        settings = json.load(f)
  except FileNotFoundError:
    print("Settings file was not found.")
  except json.JSONDecodeError:
    print("Error decoding settings JSON.")

def show_settings():
  global settings
  print(f'This is your current settings: {settings}')

def save_settings():
  global settings
  with open('settings.json', 'w') as f:
    json.dump(settings, f, indent=4)

def set_default_location():
  global settings
  default_location = input("Enter default location: ")
  if default_location:
    settings['default_location'] = default_location
    save_settings()
    print(f'Default location set to: {default_location}')

def set_temperature_units():
  global settings
  temperature_units = input("Enter temperature units (Celsius or Farenheit): ")
  if temperature_units:
    settings['temperature_units'] = temperature_units
    save_settings()
    print(f'Temperature Units set to: {temperature_units}')
    show_settings()

def add_favorite_location():
  global settings
  favorite_location = input("Enter a new favorite location: ")
  if favorite_location:
    if 'favorite_locations' in settings:
      settings['favorite_locations'].append(favorite_location)
    else:
      settings['favorite_locations'] = [favorite_location]
    save_settings()
    print(f'Favorite location added: {favorite_location}')
    show_settings()

Handle dates

In [2]:
from datetime import datetime
import pytz

def display_date_time(user_timezone, location_timezone=None):
  print("\n")
  user_time = datetime.now(pytz.timezone(user_timezone))
  formatted_user_time = user_time.strftime("%A, %B %d, %Y, %I:%M %p")
  print(f"Your current date and time in {user_timezone}: {formatted_user_time}")

  if location_timezone:
    try:
      location_timezone = pytz.timezone(location_timezone)
    except pytz.UnknownTimeZoneError:
      print(f"Invalid timezone: {location_timezone}")
      return

    location_time = user_time.astimezone(pytz.timezone(location_timezone))
    formatted_location_time = location_time.strftime("%A, %B %d, %Y, %I:%M %p")
    print(f"Date and time in {location_timezone}: {formatted_location_time}")

Handle weather fetching and display

In [3]:
import requests

OPEN_WEATHER_API_KEY = 'a77163f24622134a4f910ec858cf432e'

def get_preferred_units():
  global settings
  # 'metric' for Celsius or 'imperial' for Fahrenheit
  if 'temperature_units' in settings:
    if settings['temperature_units'] == 'Celsius':
      return 'metric'
    elif settings['temperature_units'] == 'Fahrenheit':
      return 'imperial'
  else:
    return 'metric'

def get_weather(city_name):
  base_url = "https://api.openweathermap.org/data/2.5/weather"
  params = {
      'q': city_name,
      'appid': OPEN_WEATHER_API_KEY,
      'units': get_preferred_units()
  }

  response = requests.get(base_url, params=params)

  if response.status_code == 200:
      data = response.json()
      # print(json.dumps(data, indent=4))
      return data
  else:
      return None

def display_weather_data(weather_data):
  main = weather_data['main']
  wind = weather_data['wind']
  weather_description = weather_data['weather'][0]['description']

  print(f"Temperature: {main['temp']}{'°C' if get_preferred_units() == 'Celsius' else '°F'}")
  print(f"Humidity: {main['humidity']}%")
  print(f"Pressure: {main['pressure']} hPa")
  print(f"Weather Description: {weather_description}")
  print(f"Wind Speed: {wind['speed']} m/s")

Main Menu Display And Actions

Default settings can be set by menu options.

These are the Menu Options for Weader Checking Application:

1. Display Your Current Settings
2. Set Default Location
3. Set Temperature Units
4. Add Favorite Location
5. Check Weather For Favorites - Will present a list of you favorites locations.
6. Check Weather For New or Default Location
7. Exit


In [4]:
def check_weather(city):
  global settings
  if not city:
    if 'default_location' not in settings:
      print(f"No default location found in you settings, Please set default location by menu option 1")
      return
    city = settings["default_location"]

  weather_data = get_weather(city)

  if weather_data:
    display_date_time("Israel", city)
    print("\n")
    print(f"The weather conditions in {city}:")
    display_weather_data(weather_data)
  else:
    print("Failed to retrieve data")

def check_weather_for_favorite_location():
  global settings
  locations_display = [f"{index}. {location}" for index, location in enumerate(settings.get('favorite_locations', []), start=1)]
  if locations_display:
    print("Your Favorite Locations:")
    for location in locations_display:
      print(f"{location}")
    index = input("Which location to check for weather? (Choose from the list): ")
    if index and index.isdigit and 0 < int(index) and int(index) <= len(settings['favorite_locations']):
      check_weather(settings['favorite_locations'][int(index)-1])
    else:
      print("Invalid choice")
  else:
    print("No favorite locations found in your settings")

def main_menu():
  load_settings()
  while True:
    print("\nMain Menu:")
    print("1. Your Current Settings")
    print("2. Set Default Location")
    print("3. Set Temperature Units")
    print("4. Add Favorite Location")
    print("5. Check Weather For Favorites")
    print("6. Check Weather For A New or Default Location")
    print("7. Exit")
    choice = input("Enter your choice: ")

    if choice == '1':
      show_settings()
    elif choice == '2':
      set_default_location()
    elif choice == '3':
      set_temperature_units()
    elif choice == '4':
      add_favorite_location()
    elif choice == '5':
      check_weather_for_favorite_location()
    elif choice == '6':
      city = input("Enter a city name: ")
      check_weather(city)
    elif choice == '7':
      print("See you next time.")
      break
    else:
        print("Invalid choice. Please try again.")

main_menu()

Settings file was not found.

Main Menu:
1. Your Current Settings
2. Set Default Location
3. Set Temperature Units
4. Add Favorite Location
5. Check Weather For Favorites
6. Check Weather For A New or Default Location
7. Exit
Enter your choice: 1
This is your current settings: {}

Main Menu:
1. Your Current Settings
2. Set Default Location
3. Set Temperature Units
4. Add Favorite Location
5. Check Weather For Favorites
6. Check Weather For A New or Default Location
7. Exit
Enter your choice: 3
Enter temperature units (Celsius or Farenheit): Farenheit
Temperature Units set to: Farenheit
This is your current settings: {'temperature_units': 'Farenheit'}

Main Menu:
1. Your Current Settings
2. Set Default Location
3. Set Temperature Units
4. Add Favorite Location
5. Check Weather For Favorites
6. Check Weather For A New or Default Location
7. Exit
Enter your choice: 6
Enter a city name: Greece


Your current date and time in Israel: Saturday, June 15, 2024, 10:12 AM
Invalid timezone: Greec

KeyboardInterrupt: Interrupted by user